In [11]:
import numpy as np
import torch, torch.nn as nn
import snntorch as snn
from snntorch import surrogate, utils, backprop
import snntorch.functional as SF
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt



In [2]:
import torch
x = torch.rand(5, 3)
print(x)

torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.get_device_name(0)

tensor([[0.8117, 0.5646, 0.8092],
        [0.4584, 0.8917, 0.1569],
        [0.5813, 0.7541, 0.3025],
        [0.1735, 0.0034, 0.8406],
        [0.4504, 0.0697, 0.3640]])


'NVIDIA GeForce RTX 4090'

In [3]:
V = 1
R_0 = 6 * 10 ** 5 * np.exp(-1.07*V)
alpha = 1.09*V - 0.454
V_ninf = 0.35
V_n0 = 1.03*V - 0.391
mu = -0.038 * V ** 2 + 0.142 * V - 0.057
V_scmax = 5.17 * 10 ** -2 * np.exp(2.01 * V)
t = np.arange(0, 100, 1, dtype=int)

: 

In [2]:
# dataloader arguments
batch_size = 128
data_path = "C:\\Users\\Mr_VC\\git\\mnn-torch\\data"
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")

In [3]:
# Define a transform
transform = transforms.Compose(
    [
        transforms.Resize((28, 28)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize((0,), (1,)),
    ]
)

mnist_train = datasets.MNIST(
    data_path, train=True, download=True, transform=transform
)
mnist_test = datasets.MNIST(
    data_path, train=False, download=True, transform=transform
)

training_loader = torch.utils.data.DataLoader(
    mnist_train, batch_size=batch_size, shuffle=True, drop_last=True
)
validation_loader = torch.utils.data.DataLoader(
    mnist_test, batch_size=batch_size, shuffle=True, drop_last=True
)

In [4]:
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50

In [5]:
from mnn_torch.models import MSCNN

In [25]:
# Network Architecture
num_kernels = 5
num_conv1 = 12
num_conv2 = 64
max_pooling = 2
num_hidden = num_conv2 * 4 * 4
num_outputs = 10

memrisitive_config = {
                # "experimental_data": experimental_data,
                # "k_V": 0.5,
                "ideal": True,
                "disturb_conductance": False,
                }

net = MSCNN(device=device, beta=beta, spike_grad=spike_grad, batch_size=batch_size,
            num_kernels=num_kernels, num_conv1=num_conv1, num_conv2=num_conv2,
            max_pooling=max_pooling, num_hidden=num_hidden, num_outputs=num_outputs,
            memrisitive_config=memrisitive_config)

loss_fn = SF.ce_rate_loss()

In [7]:
def forward_pass(net, num_steps, data):
  mem_rec = []
  spk_rec = []
  utils.reset(net)  # resets hidden states for all LIF neurons in net

  for step in range(num_steps):
      spk_out, mem_out = net(data)
      spk_rec.append(spk_out)
      mem_rec.append(mem_out)

  return torch.stack(spk_rec), torch.stack(mem_rec)

In [9]:
def batch_accuracy(train_loader, net, num_steps):
  with torch.no_grad():
    total = 0
    acc = 0
    net.eval()

    train_loader = iter(train_loader)
    for data, targets in train_loader:
      data = data.to(device)
      targets = targets.to(device)
      spk_rec, _ = forward_pass(net, num_steps, data)

      acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
      total += spk_rec.size(1)

  return acc/total

In [26]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2, betas=(0.9, 0.999))
num_epochs = 1
loss_hist = []
test_acc_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):

    # Training loop
    for data, targets in iter(training_loader):
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, _ = forward_pass(net, num_steps, data)

        # initialize the loss & sum over time
        loss_val = loss_fn(spk_rec, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        if counter % 50 == 0:
            with torch.no_grad():
                net.eval()

                # Test set forward pass
                test_acc = batch_accuracy(validation_loader, net, num_steps)
                print(f"Iteration {counter}, Test Acc: {test_acc * 100:.2f}%\n")
                test_acc_hist.append(test_acc.item())

        counter += 1

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same